In [ ]:
import subprocess
import os
import zipfile
import arcgis
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureLayer, FeatureCollection, FeatureSet
from arcgis.geometry.filters import intersects
from arcgis.geometry import Geometry, Point, Polyline, lengths
import csv

In [ ]:
#Logging in
# There are two ways to log in. One would require a client ID, which you might not have access to 
# unless your GIS administrator gives you the right privileges. I had trouble logging in without a client ID, but you might not. 
# Try method 1 first, especially if you don't have a client ID. If that doesn't work... 
# Try getting the right privileges from your admin and proceed with method 2.

In [ ]:
# Method 1: Logging in without a client ID
gis_url = "https://your_org.maps.arcgis.com" # url to your org's arcGIS
username = "your_username" # username to your arcGIS account
password = "your_password" # password to your arcGIS account
gis = GIS(gis_url, username, password) 
print("Logged in as: " + gis.properties.user.username)

In [ ]:
# Method 2: Logging in with a client ID
gis_url= "https://your_org.maps.arcgis.com" # url to your org's arcGIS
your_client_id = " " # paste client ID here
gis = GIS(gis_url, client_id=your_client_id)
print("Successfully logged in as: " + gis.properties.user.username)

In [ ]:
# Converts KMLs to shapefiles, zips them, and stores them in a local output folder

kml_folder = r"C:\Users\tooba\Downloads\idaho_sites_zero_buildings" #path to the folder with all your kmls
output_folder = r"C:\Users\tooba\Downloads\output_shp" #path to the folder where all the shapefiles will go

def kml_to_shapefile(kml_path, shp_path):
    command = ["ogr2ogr", "-f", "ESRI Shapefile", shp_path, kml_path]
    try:
        subprocess.run(command, check=True, capture_output=True, text=True)
        #print(f"Converted '{kml_path}' to '{shp_path}'")
    except subprocess.CalledProcessError as e:
        print(f"Error converting to shapefile: {e}")

def zip_shapefile_components(shp_path):
    folder = os.path.dirname(shp_path)
    basename = os.path.splitext(os.path.basename(shp_path))[0]
    zip_path = os.path.join(folder, basename + ".zip")

    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for ext in [".shp", ".shx", ".dbf", ".prj", ".cpg", ".qix"]:
            filepath = os.path.join(folder, basename + ext)
            if os.path.exists(filepath):
                zipf.write(filepath, os.path.basename(filepath))
                os.remove(filepath)

for file in os.listdir(kml_folder):
    if file.endswith(('.kml', '.kmz')):
        kml_path = os.path.join(kml_folder, file)
        #print(kml_path)
        basename = os.path.splitext(file)[0]
        shp_path = os.path.join(output_folder, basename + ".shp")
        #print(shp_path)
        kml_to_shapefile(kml_path, shp_path)
        zip_shapefile_components(shp_path)

print(f"Completed file conversions. Go check them out in folder: {os.path.basename(output_folder)}.")


In [ ]:
# Imports the shapefiles from your local output folder to your online ArcGIS account, converts them to feature layers, and publishes them to your webmap 

webmap_search = gis.content.search("title:ce", item_type="Web Map")
webmap_item = webmap_search[0]

webmap_data = webmap_item.get_data()

for zip_shp_file in os.listdir(output_folder):
    zip_shp_path = os.path.join(output_folder, zip_shp_file)
    # print(zip_shp_path)
    name = os.path.basename(zip_shp_file[:-4])
    # print(name)
    existing_layers = gis.content.search(f'title:"{name}"', max_items=1)
    site_exists = len(existing_layers) > 0

    if site_exists:
        print(f"{name} already exists. Skipping.")
        continue
    else:
        item_properties = {"title": str(name), "type": "Shapefile", "tags": "shapefile, layer, ce" } 
        shapefile_item = gis.content.add(item_properties, data=zip_shp_path)
        published_layer = shapefile_item.publish()
        shp_url = published_layer.layers[0].url
        
        # add to the web map's operational layers
        shp_layer = {"id": shp_url,"title": name,"url": shp_url,"visibility": True,"opacity": 1,"layerType": "ArcGISFeatureLayer"}
        webmap_data["operationalLayers"].append(shp_layer)
        print(f"Added {name} to web map.")
        webmap_item.update(data=webmap_data)
    
print(f"Web map '{webmap_item.title}' successfully updated with CE sites!")